<a href="https://colab.research.google.com/github/anshumansharma0512/Deep-Learning-Lab/blob/main/CIPHER10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.model_selection import ParameterGrid

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

# Load CIFAR-10 dataset
train_data = datasets.CIFAR10(root="data", train=True, transform=transform, download=True)
test_data = datasets.CIFAR10(root="data", train=False, transform=transform, download=True)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Hyperparameter grid for custom CNN
param_grid = {
    "activation": [nn.ReLU, nn.Tanh, nn.LeakyReLU],
    "weight_init": ["xavier", "kaiming", "random"],
    "optimizer": ["sgd", "adam", "rmsprop"]
}

# Define CNN model
class CNN(nn.Module):
    def __init__(self, activation):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)  # CIFAR-10 has 10 classes
        self.activation = activation()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.activation(self.fc1(x))
        x = self.fc2(x)  # No softmax here because CrossEntropyLoss applies it internally
        return x

# Weight initialization
def initialize_weights(model, init_type):
    for layer in model.modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            if init_type == "xavier":
                nn.init.xavier_uniform_(layer.weight)
            elif init_type == "kaiming":
                nn.init.kaiming_uniform_(layer.weight, nonlinearity="relu")
            elif init_type == "random":
                nn.init.uniform_(layer.weight, a=-0.1, b=0.1)

# Training and evaluation for custom CNN
def train_and_evaluate_cnn(params):
    activation = params["activation"]
    weight_init = params["weight_init"]
    optimizer_type = params["optimizer"]

    model = CNN(activation).to(device)
    initialize_weights(model, weight_init)

    criterion = nn.CrossEntropyLoss()
    if optimizer_type == "sgd":
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    elif optimizer_type == "adam":
        optimizer = optim.Adam(model.parameters(), lr=0.001)
    elif optimizer_type == "rmsprop":
        optimizer = optim.RMSprop(model.parameters(), lr=0.001)

    # Training loop
    for epoch in range(5):
        model.train()
        train_loss = 0
        correct = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

        accuracy = correct / len(train_data)
        print(f"[CNN] Epoch {epoch + 1}: Loss = {train_loss / len(train_loader):.4f}, "
              f"Accuracy = {accuracy:.4f}")

    # Evaluate on the test set
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    accuracy = correct / len(test_data)
    print(f"[CNN] Test Loss = {test_loss / len(test_loader):.4f}, Test Accuracy = {accuracy:.4f}")
    return accuracy, model

# Hyperparameter tuning loop
best_cnn_accuracy = 0
best_cnn_model = None
for params in ParameterGrid(param_grid):
    print(f"Testing CNN combination: {params}")
    accuracy, model = train_and_evaluate_cnn(params)
    if accuracy > best_cnn_accuracy:
        best_cnn_accuracy = accuracy
        best_cnn_model = model
        torch.save(model.state_dict(), "best_model_cifar10.pth")

# Fine-tune ResNet-18
resnet18 = models.resnet18(pretrained=True)
num_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_features, 10)  # CIFAR-10 has 10 classes
resnet18 = resnet18.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=0.001)

# Train ResNet-18
def train_resnet(model, train_loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        train_loss = 0
        correct = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

        accuracy = correct / len(train_data)
        print(f"[ResNet-18] Epoch {epoch + 1}: Loss = {train_loss / len(train_loader):.4f}, "
              f"Accuracy = {accuracy:.4f}")

# Evaluate ResNet-18
def evaluate_resnet(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()

    accuracy = correct / len(test_data)
    print(f"[ResNet-18] Test Loss = {test_loss / len(test_loader):.4f}, Test Accuracy = {accuracy:.4f}")
    return accuracy

# Train and evaluate ResNet-18
train_resnet(resnet18, train_loader, optimizer, criterion, epochs=5)
resnet18_accuracy = evaluate_resnet(resnet18, test_loader, criterion)

# Save ResNet-18
torch.save(resnet18.state_dict(), "resnet18_cifar10.pth")

# Comparison
print(f"Best CNN Test Accuracy: {best_cnn_accuracy:.4f}")
print(f"ResNet-18 Test Accuracy: {resnet18_accuracy:.4f}")

if resnet18_accuracy > best_cnn_accuracy:
    print("ResNet-18 performed better!")
else:
    print("The best CNN performed better!")


100%|██████████| 170M/170M [00:05<00:00, 31.0MB/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
Testing CNN combination: {'activation': <class 'torch.nn.modules.activation.ReLU'>, 'optimizer': 'sgd', 'weight_init': 'xavier'}
[CNN] Epoch 1: Loss = 1.4351, Accuracy = 0.4822
[CNN] Epoch 2: Loss = 1.0236, Accuracy = 0.6378
[CNN] Epoch 3: Loss = 0.8345, Accuracy = 0.7065
[CNN] Epoch 4: Loss = 0.6896, Accuracy = 0.7587
[CNN] Epoch 5: Loss = 0.5543, Accuracy = 0.8052
[CNN] Test Loss = 0.8580, Test Accuracy = 0.7193
Testing CNN combination: {'activation': <class 'torch.nn.modules.activation.ReLU'>, 'optimizer': 'sgd', 'weight_init': 'kaiming'}
[CNN] Epoch 1: Loss = 1.3430, Accuracy = 0.5182
[CNN] Epoch 2: Loss = 0.9432, Accuracy = 0.6687
[CNN] Epoch 3: Loss = 0.7635, Accuracy = 0.7326
[CNN] Epoch 4: Loss = 0.6290, Accuracy = 0.7796
[CNN] Epoch 5: Loss = 0.5031, Accuracy = 0.8231
[CNN] Test Loss = 0.8965, Test Accuracy = 0.7129
Testing CNN combination: {'activation': <class 'torch.nn.modules.activation.Re

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 186MB/s]


[ResNet-18] Epoch 1: Loss = 0.9594, Accuracy = 0.6754
[ResNet-18] Epoch 2: Loss = 0.6578, Accuracy = 0.7797
[ResNet-18] Epoch 3: Loss = 0.5225, Accuracy = 0.8242
[ResNet-18] Epoch 4: Loss = 0.4078, Accuracy = 0.8640
[ResNet-18] Epoch 5: Loss = 0.3338, Accuracy = 0.8880
[ResNet-18] Test Loss = 0.6303, Test Accuracy = 0.8024
Best CNN Test Accuracy: 0.7293
ResNet-18 Test Accuracy: 0.8024
ResNet-18 performed better!
